In [1]:
import pandas as pd
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import random
import torch
from stable_baselines3 import PPO


In [ ]:
# Define the environment
class CPUSchedulingEnv(gym.Env):
    def __init__(self, dataset):
        super(CPUSchedulingEnv, self).__init__()
        self.dataset = dataset
        self.action_space = spaces.Discrete(len(dataset))
        self.observation_space = spaces.Box(
            low=0, high=1, shape=(len(dataset[0]),), dtype=np.float32
        )
        self.reset()

    def reset(self):
        self.current_time = 0
        self.processes = sorted(
            self.dataset, key=lambda x: x[2]
        )  # Sort by arrival time
        self.completed_processes = []
        self.current_index = 0
        return self._get_state()

    def _get_state(self):
        # Normalized state representation (e.g., scaled attributes)
        return np.array(self.processes[self.current_index]) / np.array(
            [999, 300, 1, 1, 10]
        )

    def step(self, action):
        # Execute the selected process
        selected_process = self.processes[action]
        burst_time = selected_process[1]
        self.current_time += burst_time
        self.completed_processes.append(selected_process)

        reward = -burst_time  # Example reward function (minimize burst time)
        self.current_index += 1
        done = self.current_index >= len(self.processes)
        next_state = (
            self._get_state() if not done else np.zeros(self.observation_space.shape)
        )
        return next_state, reward, done, {}

In [3]:
data = pd.read_csv("process_data.csv")

In [4]:
data.head()

,Job Id,Burst time,Arrival Time,Prremptive,Resources
0,247,199,0.4100,0,8
1,29,193,0.5925,1,2
2,170,75,0.3600,1,4
3,164,42,0.9725,0,8
4,312,257,0.6125,0,4


In [ ]:
env = CPUSchedulingEnv(data)